In [ ]:
import imp
import _config
imp.reload(_config)
imp.reload(z2_save_jaspar)

In [3]:
import pandas as pd
import numpy as np

import z1_save_oligos, z2_save_jaspar
oligos, oligos_by_exp = z1_save_oligos.load_oligos()
jaspar = z2_save_jaspar.load_jaspar()


wt_oligos = oligos.loc[lambda df: df.mutant_num == 0]
starts = wt_oligos.starts.sort_values()
diff_prev = pd.Series( (starts - np.roll(starts,1)).values,index = starts.values)
diff_nex = pd.Series((starts - np.roll(starts,-1)).values,index = starts.values)
firsts = diff_prev.loc[lambda df: abs(df) > 30]
lasts = diff_nex.loc[lambda df: abs(df) > 30]

#LOADS BIOLOGICAL MOTIFS AND SCANS ALL SUBREGIONS FOR OCCURENCES
from pyfaidx import Fasta
sequences_fa = Fasta('/Users/ben/genomes/GRCh38.primary_assembly.genome.fa')
chrseq = str(sequences_fa["chr22"])
region_bounds=[ 38699734, 39291007]
chrreg = chrseq[region_bounds[0]:region_bounds[1]]
intervals = pd.concat([pd.Series(firsts.index.values).rename("start"), pd.Series(lasts.index.values + 120).rename("end")],axis =1)
intervals["seq"] = intervals.apply(lambda x:chrseq[ region_bounds[0]+ x.start: region_bounds[0] +x.end], axis = 1)

import Bio
from Bio.Alphabet import IUPAC
seqseq = intervals.seq.apply(lambda x: Bio.Seq.Seq(x,alphabet = Bio.Alphabet.IUPAC.IUPACUnambiguousDNA))




In [15]:
seqseq = intervals.seq.apply(lambda x: Bio.Seq.Seq(x,alphabet = Bio.Alphabet.IUPAC.IUPACUnambiguousDNA()))

In [18]:
pwm_r60_grid = jaspar.apply(lambda  j: seqseq.apply(lambda x: list( j.pssm.search(x,threshold = j.threshold_patser))),axis=1)


In [23]:
intervals.to_csv('../out/0729_intervals.csv')

In [21]:

interval_motif_hits = pwm_r60_grid.copy()
interval_motif_hits.columns = interval_motif_hits.columns.rename("interval_id")
interval_motif_hits.index.name = "jaspar_id"
interval_motif_hits_series = interval_motif_hits.unstack()
motif_hits = interval_motif_hits_series.\
    groupby(level =["interval_id", "jaspar_id"]).\
    apply(lambda g: pd.Series({intervals.loc[g.name[0]].start + e[0] 
                                         if e[0] > 0 
                                         else intervals.loc[g.name[0]].end + e[0]:e[1]
                     for r in g for e in r}))
motif_hits_over_threshold = motif_hits.groupby(level="jaspar_id").apply(lambda x: x  - jaspar.loc[x.name].threshold_patser).loc[lambda x: x > 0]
motif_hits_over_threshold.index.levels[2].rename("position", inplace=True)
motif_info = motif_hits_over_threshold.rename("score").reset_index()[["jaspar_id","position","score"]]
tmp = motif_info.join(jaspar[["consensus","len","name"]],on = "jaspar_id")
tmp["seq_actual"] = tmp.apply(lambda x: chrreg[x.position:x.position+x["len"]],axis=1)


## CATEGORIZE OLIGO <--> MOTIF EDGES ##
Given a set of all oigos, query for all oligos which either overlap them in wildtype, or as benign / ablating 

In [ ]:
#this code is a little clunky because we're trying to work with indexed queries only ...
oligos["mutant_position"] = oligos.apply(lambda x: x.starts + x.mutant_num  * 30 if x.mutant_num > 0 else np.nan, axis =1)
oligos_by_start_mutant_start = oligos.reset_index().set_index(["starts","mutant_position"]).sort_index(level = ["starts","mutant_position"])
tmp.index.rename("motif_hit_idx",inplace=True)
wt_oligos_by_start = oligos.loc[lambda df: df.mutant_num == 0].reset_index().set_index("starts").sort_index()
idx = pd.IndexSlice


In [ ]:

print("WT")
#2d grid representations of relationships b/w motifs & oligos
overlapping_wt_grid = tmp.\
    apply(lambda x : pd.Series(1,
                               index= wt_oligos_by_start.loc[idx[x.position - 180+x["len"]:x.position -30],:].oligo)
          ,axis=1)
print("AB")
ablation_grid = tmp.\
    apply(lambda x : pd.Series(1,
                               index= oligos_by_start_mutant_start.loc[idx[x.position  - 180+x["len"]:x.position ,x.position-29:x.position+x["len"]-1],:].oligo)
          ,axis=1)
print("BE")
mutants_before= \
    tmp.\
    apply(lambda x : pd.Series(1,
                               index= oligos_by_start_mutant_start.loc[idx[x.position  - 180+x["len"]:x.position ,:x.position-30],:].oligo)
          ,axis=1)
mutants_after= \
    tmp.\
    apply(lambda x : pd.Series(1,
                               index= oligos_by_start_mutant_start.loc[idx[x.position  - 180+x["len"]:x.position ,x.position+x["len"]:],:].oligo)
          ,axis=1)

print("UNSTACKING")
overlapping_wildtypes = overlapping_wt_grid.unstack().dropna()
benign_mutants = pd.concat([mutants_before.unstack().dropna(),mutants_after.unstack().dropna()]).sort_index()
ablation_mutants = ablation_grid.unstack().dropna()

In [ ]:
print("HI")

In [ ]:
#merge hit data
motif_oligos = pd.concat([overlapping_wildtypes.rename("is_overlapping_wt"),
           ablation_mutants.rename("is_ablation_mut"),
           benign_mutants.rename("is_benign_mut")],axis=1)
motif_oligos_data=motif_oligos.reset_index().set_index("motif_hit_idx").join(motif_info).reset_index()
motif_oligos_data.index.rename("motif_oligo_idx", inplace = True)
motif_oligos_data = motif_oligos_data.reset_index().set_index("oligo").join(oligos[["mu"]]).reset_index().set_index("motif_oligo_idx")

#create an annotation column "hit_type" categorizing motif oligo matches according to three types
motif_oligos_data =motif_oligos_data[["is_ablation_mut","is_overlapping_wt","is_benign_mut"]].reset_index().\
    melt(id_vars=["motif_oligo_idx"],var_name="hit_type").dropna().\
    set_index("motif_oligo_idx")[["hit_type"]].\
    join(motif_oligos_data)


motif_oligos_data["seq_actual"] = motif_oligos_data.join(tmp[["seq_actual"]],on="motif_hit_idx")["seq_actual"]

In [ ]:
#motif_oligos_data.to_csv("../out/0722_motif_oligos_data.csv")

In [15]:
motif_oligos_data = pd.read_csv("../out/0722_motif_oligos_data.csv")

In [ ]:
len(q95_starts)

In [ ]:

q95_oligos = oligos.loc[lambda df: df.starts.isin(oligos.loc[ lambda x:x.mu > oligos.mu.quantile(.98)].starts)].index
q05_oligos = oligos.loc[lambda df: df.starts.isin(oligos.loc[ lambda x:x.mu < oligos.mu.quantile(.02)].starts)].index

In [ ]:

q99_oligos = oligos.loc[lambda df: df.starts.isin(oligos.loc[ lambda x:x.mu > oligos.mu.quantile(.99)].starts)].index
q01_oligos = oligos.loc[lambda df: df.starts.isin(oligos.loc[ lambda x:x.mu < oligos.mu.quantile(.01)].starts)].index

In [ ]:
q95_starts = set(oligos.loc[q95_oligos].starts)
q05_starts = set(oligos.loc[q05_oligos].starts)
q99_starts = set(oligos.loc[q99_oligos].starts)
q01_starts = set(oligos.loc[q01_oligos].starts)

In [ ]:
pwm_score_cutoff = 6

q95_wt_hits = motif_oligos_data.\
    loc[lambda df: df.hit_type== "is_overlapping_wt"].\
    loc[lambda df: df.oligo.isin(oligos.loc[lambda x: x.starts.isin(oligos.loc[q95_oligos].starts.unique())].index)].\
    loc[lambda df:df.score>pwm_score_cutoff]

q05_wt_hits = motif_oligos_data.\
    loc[lambda df: df.hit_type== "is_overlapping_wt"].\
    loc[lambda df: df.oligo.isin(oligos.loc[lambda x: x.starts.isin(oligos.loc[q05_oligos].starts.unique())].index)].\
    loc[lambda df:df.score>pwm_score_cutoff]


In [ ]:

q99_wt_hits = motif_oligos_data.\
    loc[lambda df: df.hit_type== "is_overlapping_wt"].\
    loc[lambda df: df.oligo.isin(oligos.loc[lambda x: x.starts.isin(oligos.loc[q99_oligos].starts.unique())].index)].\
    loc[lambda df:df.score>pwm_score_cutoff]

q01_wt_hits = motif_oligos_data.\
    loc[lambda df: df.hit_type== "is_overlapping_wt"].\
    loc[lambda df: df.oligo.isin(oligos.loc[lambda x: x.starts.isin(oligos.loc[q01_oligos].starts.unique())].index)].\
    loc[lambda df:df.score>pwm_score_cutoff]


In [ ]:

jaspar_all_position_counts = motif_oligos_data.loc[lambda x: x.score> pwm_score_cutoff].groupby("jaspar_id").apply(lambda g:g.position.nunique())

jaspar_q95_instance_counts = q95_wt_hits.groupby("jaspar_id").apply(lambda g:g.position.nunique())
jaspar_q05_position_counts = q05_wt_hits.groupby("jaspar_id").apply(lambda g:g.position.nunique())

q95_jaspar_ids = q95_wt_hits.jaspar_id.unique()
q05_jaspar_ids = q05_wt_hits.jaspar_id.unique()

q99_jaspar_ids = q99_wt_hits.jaspar_id.unique()
q01_jaspar_ids = q01_wt_hits.jaspar_id.unique()
                                                    
                                                                        

In [ ]:
q95_start_motifs = motif_oligos_data.loc[motif_oligos_data.join(oligos[["starts"]],on="oligo").starts.isin(q95_starts)]
q05_start_motifs = motif_oligos_data.loc[motif_oligos_data.join(oligos[["starts"]],on="oligo").starts.isin(q05_starts)]

q99_start_motifs = motif_oligos_data.loc[motif_oligos_data.join(oligos[["starts"]],on="oligo").starts.isin(q99_starts)]
q01_start_motifs = motif_oligos_data.loc[motif_oligos_data.join(oligos[["starts"]],on="oligo").starts.isin(q01_starts)]




In [ ]:
motif_counts = pd.concat([
    q95_start_motifs.drop_duplicates(["jaspar_id","position"]).jaspar_id.value_counts().rename("count_top5"),
    q05_start_motifs.drop_duplicates(["jaspar_id","position"]).jaspar_id.value_counts().rename("count_bot5"),
    
    q99_start_motifs.drop_duplicates(["jaspar_id","position"]).jaspar_id.value_counts().rename("count_top1"),
    q01_start_motifs.drop_duplicates(["jaspar_id","position"]).jaspar_id.value_counts().rename("count_bot1"),
    
    motif_oligos_data.drop_duplicates(["jaspar_id","position"]).jaspar_id.value_counts().rename("count_all"),

],axis=1).fillna(0)

motif_counts.index.rename("jaspar_id",inplace = True)


motif_counts["frac_top5"] = motif_counts.apply(lambda x: x.count_top5 / x.count_all, axis =1)
motif_counts["frac_bot5"] = motif_counts.apply(lambda x: x.count_bot5 / x.count_all, axis =1)

motif_counts["frac_top1"] = motif_counts.apply(lambda x: x.count_top5 / x.count_all, axis =1)
motif_counts["frac_bot1"] = motif_counts.apply(lambda x: x.count_bot5 / x.count_all, axis =1)

motif_counts["diff_top_bottom"] = motif_counts.apply(lambda x: x.count_top5 -  x.count_bot5 , axis =1)



In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

### Q: Looking at all oligos, is there a systematic change in expressionl level based upon motif mutation types? ###

In [ ]:

modata_min_5 = motif_oligos_data.loc[motif_oligos_data.score > 3]
modata = modata_min_5
bins = np.arange(0,50,5)

plt.hist([modata.loc[lambda df:df.hit_type=="is_benign_mut"].mu.rename("benign"),
          modata.loc[lambda df:df.hit_type=="is_overlapping_wt"].mu.rename("overlap"),
          modata.loc[lambda df:df.hit_type=="is_ablation_mut"].mu.rename("ablation")]
         ,bins = bins, density=True,label=["b","o","ab"])

plt.legend()
ax = plt.gca()
ax.set_xlim(0,100)
ax.set_yscale("log")

A: Visually, yes, probably. Looking at oligos having a motif score of 7 or greater, there appear to be  more benign and overlapping WT oligos falling in the top mu percentiles, 25+.

### Q: Looking at only those oligos which are expressed within q95, does the above pattern repeat? ###

In [ ]:

modata_q95 = modata_min_5.loc[lambda df: df.oligo.isin(q95_oligos)]

these_data = modata_q95
bins = np.arange(0,50,5)

plt.hist([these_data.loc[lambda df:df.hit_type=="is_benign_mut"].mu.rename("benign"),
          these_data.loc[lambda df:df.hit_type=="is_overlapping_wt"].mu.rename("overlap"),
          these_data.loc[lambda df:df.hit_type=="is_ablation_mut"].mu.rename("ablation")]
         ,bins = bins, density=True,label=["b","o","ab"])

plt.legend()
ax = plt.gca()
ax.set_xlim(0,100)
ax.set_yscale("log")

A: Well, yes. But this is pretty obvious. It's where we were looking before.

### Q: Switching to a motif-focused view, can we identify a set of N motifs which have a small number of high-scoring hits, all of which are highly expressed in wildtype oligos? ###

In [ ]:

#bins = np.arange(0,200,5)
ax = plt.gca()
ax.set_xlabel("number of occurences")
ax.set_ylabel("count of motifs in range")

ax.set_title("occurence counts of motifs in highly expressed oligos and all oligos")

h = plt.hist(jaspar_expressed_position_counts,density = True, label = "highly expressed")
plt.figure()
h = plt.hist(jaspar_all_position_counts ,density = True, label = "all")



In [ ]:
this_data

In [ ]:
this_data.diff_top_bottom.sort_values().index

In [ ]:
this_data

In [ ]:
f = plt.gcf()
f.set_size_inches(12,4)

this_data = motif_counts.loc[q95_jaspar_ids]
this_data["diffrank"] = (this_data.frac_top5 - this_data.frac_bot5).sort_values().argsort()
this_data2= this_data.reset_index().melt(value_name="val", var_name="pool",
                                         value_vars = ["frac_top5","frac_bot5"], 
                                         id_vars=["jaspar_id","diffrank","diff_top_bottom","count_top5"])
sns.scatterplot( x = "diffrank", y="val", hue = "pool",size="count_top5", data = this_data2)

ax =plt.gca()
ax.set_title("Fraction of motif instances in top 5% and bottom 5% of oligos")
ax.set_xlabel("JASPAR motifs, sorted by representation in top & bottom 5% of WT oligos")


jaspar_mu_types = motif_oligos_data.groupby(["jaspar_id","hit_type"]).mu.mean().unstack()
this_data3 = this_data2.join(jaspar_mu_types, on="jaspar_id")
this_data3 = this_data3.melt(  
    value_name = "mu_val",
    var_name="mu_type",
    value_vars = ["is_ablation_mut", "is_benign_mut", "is_overlapping_wt"],
    id_vars = ["val","pool","jaspar_id","diffrank","diff_top_bottom","count_top5"])


f = plt.figure()
ax = plt.gca()
f.set_size_inches(12,4)

ax.set_title("Expresson rates of motif-ablating and motif-preserving mutant oligos")
sns.scatterplot(x= "diffrank",y ="mu_val", hue = "mu_type", data = this_data3)
ax.set_xlabel("JASPAR motifs, sorted by representation in top & bottom 5% of WT oligos")

In [ ]:
f = plt.gcf()
f.set_size_inches(12,4)

this_data = motif_counts.loc[q99_jaspar_ids]
this_data["diffrank"] = (this_data.frac_top1 - this_data.frac_bot1).sort_values().argsort()
this_data2= this_data.reset_index().melt(value_name="val", var_name="pool",
                                         value_vars = ["frac_top1","frac_bot1"], 
                                         id_vars=["jaspar_id","diffrank","diff_top_bottom","count_top1"])
sns.scatterplot( x = "diffrank", y="val", hue = "pool",size="count_top1", data = this_data2)

ax =plt.gca()
ax.set_title("Fraction of motif instances in top 1% and bottom 1% of oligos")
ax.set_xlabel("JASPAR motifs, sorted by representation in top & bottom 1% of WT oligos")


jaspar_mu_types = motif_oligos_data.groupby(["jaspar_id","hit_type"]).mu.mean().unstack()
this_data3 = this_data2.join(jaspar_mu_types, on="jaspar_id")
this_data3 = this_data3.melt(  
    value_name = "mu_val",
    var_name="mu_type",
    value_vars = ["is_ablation_mut", "is_benign_mut", "is_overlapping_wt"],
    id_vars = ["val","pool","jaspar_id","diffrank","diff_top_bottom","count_top1"])


f = plt.figure()
ax = plt.gca()
f.set_size_inches(12,4)

ax.set_title("Expresson rates of motif-ablating and motif-preserving mutant oligos")
sns.scatterplot(x= "diffrank",y ="mu_val", hue = "mu_type", data = this_data3)
ax.set_xlabel("JASPAR motifs, sorted by representation in top & bottom 5% of WT oligos")

In [ ]:

with_effect_size = this_data.join(jaspar_mu_types, on ="jaspar_id")
with_effect_size["effect_size"] = with_effect_size["is_ablation_mut"] - with_effect_size[["is_benign_mut","is_overlapping_wt"]].mean(axis=1) 
selected =with_effect_size.loc[lambda x:np.abs( x.effect_size) > .5]
selected.join(jaspar)[["name"]]


In [ ]:
out_722_lin = selected[["count_top5","count_bot5"]].\
    join(this_data7).\
    join(jaspar[["name","consensus"]]).\
    rename({"delta_HCT":"ablation_effect_size_HCT116",
           "delta_OTHER":"ablation_effect_size_NON_HCT116",
           "count_top5":"motif_incidence_top_oligos",
        "count_bot5":"motif_incidence_negative_controls",
           },axis="columns")

In [ ]:
out_722_lin.to_csv("../out/0722_lin_email_motifs_whole_region_search.csv")

In [ ]:
mdata = motif_oligos_data.loc[lambda df: df.score > pwm_score_cutoff]


In [ ]:
all_motifs_with_experiments = motif_oligos_data.join(oligos_by_exp[["exp_norm_mu"]].reset_index(level=0),on="oligo").\
    groupby(["jaspar_id","hit_type","exp"])["exp_norm_mu"].mean().\
    reorder_levels(["jaspar_id","exp","hit_type"]).unstack()

In [ ]:
all_motifs_with_experiments["delta"] =   all_motifs_with_experiments["is_ablation_mut"] - all_motifs_with_experiments[["is_benign_mut","is_overlapping_wt"]].mean(axis=1)




In [ ]:
import re

this_data6 = all_motifs_with_experiments.reset_index("exp")
this_data6["rep"] = this_data6["exp"].apply(lambda x:int(re.compile("BR([12])").search(x).group(1)))
this_data6["exp_nm"] = this_data6["exp"].apply(lambda x:re.compile("(.*)_BR([12])").search(x).group(1))


In [ ]:
f,subs = plt.subplots(1,3)
f.set_size_inches(14,5)
plt.sca(subs[0])
this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm=="DLD1_WT"].groupby("jaspar_id").delta.mean().rename("delta_DLD1")
    ],axis=1)


ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & DLD1")
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])


sns.scatterplot(x = "delta_HCT", y="delta_DLD1", data = this_data7)


plt.sca(subs[1])

this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm=="U2OS_WT"].groupby("jaspar_id").delta.mean().rename("delta_U2OS")
    ],axis=1)

ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & U2OS")
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

sns.scatterplot(x = "delta_HCT", y="delta_U2OS", data = this_data7)
plt.sca(subs[2])

this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="DLD1_WT"].groupby("jaspar_id").delta.mean().rename("delta_DLD1"),
     this_data6.loc[lambda x:x.exp_nm=="U2OS_WT"].groupby("jaspar_id").delta.mean().rename("delta_U2OS")
    ],axis=1)

ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in DLD1 & U2OS")

sns.scatterplot(x = "delta_DLD1", y="delta_U2OS", data = this_data7)

ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

plt.tight_layout()

In [ ]:
f,subs = plt.subplots(2,2)
f.set_size_inches(9,9)
plt.sca(subs[1][1])
this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm!="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_OTHER")
    ],axis=1)


ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & OTHER CELL TYPES")


sns.scatterplot(x = "delta_HCT", y="delta_OTHER", data = this_data7)


plt.sca(subs[1][0])

this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm=="HCT116_Gemcitabine2uM"].groupby("jaspar_id").delta.mean().rename("delta_HCT_GEM")
    ],axis=1)

ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & HCT116 GEM")

sns.scatterplot(x = "delta_HCT", y="delta_HCT_GEM", data = this_data7)



plt.sca(subs[0][0])

this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="U2OS_NFKB1KO_Sorted_GFPpos"].groupby("jaspar_id").delta.mean().rename("delta_NFKB1_pos"),
     this_data6.loc[lambda x:x.exp_nm=="U2OS_NFKB1KO_Sorted_GFPneg"].groupby("jaspar_id").delta.mean().rename("delta_NFKB1_neg")
    ],axis=1)

ax = plt.gca()
ax.set_title("NFKB1 KO, GFP Pos vs Neg")

sns.scatterplot(x = "delta_NFKB1_pos", y="delta_NFKB1_neg", data = this_data7)



plt.sca(subs[0][1])

this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="U2OS_WT"].groupby("jaspar_id").delta.mean().rename("delta_U2OS"),
     this_data6.loc[lambda x:x.exp_nm=="U2OS_NFKB1KO_Sorted_GFPneg"].groupby("jaspar_id").delta.mean().rename("delta_U2OS_NFKB1")
    ],axis=1)

ax = plt.gca()
ax.set_title("U2OS Wildtype vs. NFKB1 KO")
sns.scatterplot(x = "delta_U2OS", y="delta_U2OS_NFKB1", data = this_data7)
plt.tight_layout()



In [ ]:
this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm!="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_OTHER")
    ],axis=1)


ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & OTHER CELL TYPES")


sns.scatterplot(x = "delta_HCT", y="delta_OTHER", hue = "frac_top5", data = motif_counts.join(this_data7))


### Standard deviations change very much ###

In [ ]:
plt.scatter(this_data6.groupby("jaspar_id").delta.std(),this_data6.groupby("jaspar_id").delta.mean())

In [ ]:
jaspar_exp_data = mdata.join(oligos_by_exp[["exp_norm_mu"]].reset_index(level=0),on="oligo").\
    groupby(["jaspar_id","hit_type","exp"])["exp_norm_mu"].mean().\
    reorder_levels(["jaspar_id","exp","hit_type"]).unstack()

In [ ]:
jaspar_exp_delta =   jaspar_exp_data["is_ablation_mut"] - jaspar_exp_data[["is_benign_mut","is_overlapping_wt"]].mean(axis=1)

In [ ]:
import re

In [ ]:

this_data4 = top1_mutypes.join(jaspar_exp_delta.rename("delta").reset_index().set_index("jaspar_id"),on="jaspar_id")
this_data4["rep"] = this_data4["exp"].apply(lambda x:int(re.compile("BR([12])").search(x).group(1)))
this_data4["exp_nm"] = this_data4["exp"].apply(lambda x:re.compile("(.*)_BR([12])").search(x).group(1))


In [ ]:
f = plt.gcf()
f.set_size_inches(10,5)


sns.scatterplot(x = "diffrank",y ="delta", hue = "exp",data = this_data4)

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax = plt.gca()
ax.set_title("effects of ablative mutations on oligo expression in different cell types")
ax.set_ylabel("motif effect\n ABLATED expression vs. mean of WILDTYPE & BENIGN)")

f = plt.figure()
f.set_size_inches(10,5)
sns.lineplot(x = "diffrank", y = "delta", hue = "exp_nm", data = this_data4.loc[lambda x:x.diffrank>25])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.tight_layout()


In [ ]:
this_data4a = this_data4.join(selected.apply(lambda x:"selected",axis=1).rename("is_selected"),on="jaspar_id")
this_data4a.is_selected.fillna("not",inplace=True)
sns.scatterplot( x = "diffrank",y="delta", hue = "is_selected", data = this_data4a)

In [ ]:
this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm!="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_OTHER")
    ],axis=1)


ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & OTHER CELL TYPES")


sns.scatterplot(x = "delta_HCT", y="delta_OTHER",alpha=.2,color="lightgray", data = motif_counts.join(this_data7))


this_data7 = pd.concat(
    [this_data6.loc[lambda x:x.exp_nm=="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_HCT"),
     this_data6.loc[lambda x:x.exp_nm!="HCT116_WT"].groupby("jaspar_id").delta.mean().rename("delta_OTHER")
    ],axis=1)


ax = plt.gca()
ax.set_title("visualizing expression change ablated\n motifs in HCT116 & OTHER CELL TYPES")


sns.scatterplot(x = "delta_HCT", y="delta_OTHER", hue = "is_selected", data = motif_counts.join(this_data7).join(this_data4a.set_index("jaspar_id")[["is_selected"]],on="jaspar_id"))
